Análisis de modelos con TensorFlow Model Analysis
=================================================

In [1]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix 
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix 

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
2021-10-11 12:57:23.313182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-11 12:57:23.313235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: OK


In [9]:
import tensorflow as tf
import tensorflow_model_analysis as tfma

Primero necesitamos convertir nuestro modelo a un formato de evaluación:

In [3]:
eval_shared_model = tfma.default_eval_shared_model(eval_saved_model_path='./datasets/uci_census/model/1',
                                                   tags=[tf.saved_model.SERVING])

Luego, debemos indicarle a TFMA cual es nuestra variable objetivo junto con algunas especificaciones de como queremos evaluar el modelo, utilizando cuales características y computando cuales métricas.

In [4]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(model_type=tfma.constants.TF_KERAS, 
                                label_key='Target',
                                signature_name='classification')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='BinaryAccuracy', 
                              threshold=tfma.MetricThreshold(
                                  value_threshold=tfma.GenericValueThreshold(
                                      lower_bound={'value': 0.6}),
                                      change_threshold=tfma.GenericChangeThreshold(
                                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                          absolute={'value': -1e-10}))),
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
        ])
    ])

Una vez que tenemos configuramos el modelo, lo evaluamos

In [6]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    data_location='datasets/uci_census/adult.tfrecord',
    output_path='datasets/uci_census/eval.tfrecord',
    file_format='tfrecords')

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).


Visualizamos los resultados en Jupyter:

In [10]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…